<a href="https://colab.research.google.com/github/Vakhranev/Gramota/blob/main/%D0%A1%D1%80%D0%B0%D0%B2%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5%20RAG%20%D0%B8%20Correct%20Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas sentence-transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [6]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Загрузка данных из CSV
file_path = 'Cleaned_RAG_ANSWERS.csv'  # Укажи путь к твоему файлу
df = pd.read_csv(file_path)

# Загрузка предобученной модели для семантического анализа
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Вычисление эмбеддингов для правильных ответов и ответов нейронки
correct_embeddings = model.encode(df['Correct_Answer_Cleaned'].tolist(), convert_to_tensor=True)
rag_embeddings = model.encode(df['RAG_ANSWERS'].tolist(), convert_to_tensor=True)

# Вычисление косинусного сходства между правильными ответами и ответами нейронки
similarity_scores = util.pytorch_cos_sim(correct_embeddings, rag_embeddings)

# Извлечение диагональных значений, которые представляют сходство между соответствующими парами
df['Semantic_Similarity'] = similarity_scores.diag().cpu().numpy()

# Сохранение результатов в новый CSV файл
df.to_csv('Semantic_Similarity_Results.csv', index=False)

# Вывод первых нескольких строк с результатами
print(df[['Question', 'Correct_Answer_Cleaned', 'RAG_ANSWERS', 'Semantic_Similarity']].head())

                                            Question  \
0  Здравствуйте! Как пишется слово НЕ_ВИДИМЫЙ в д...   
1  Со строчной или заглавной? "В городе проходит ...   
2  <p>Здравствуйте. Нужно ли выделять сорт растен...   
3  <p>Добрый день. В слове молоденький суффикс -е...   
4  Скажите, пожалуйста, можно ли говорить «смерте...   

                              Correct_Answer_Cleaned  \
0  Правильно слитное написание: Над головой хрипл...   
1  В данном случае всероссийская пишется со строч...   
2  Разница в типах текстов: такие названия не выд...   
3  Вы путаете правило о написании суффиксов -еньк...   
4  В терминологическом значении правильно только ...   

                                         RAG_ANSWERS  Semantic_Similarity  
0  В данном контексте слово "НЕ_ВИДИМЫЙ" пишется ...             0.975246  
1  В данном контексте "всероссийская" пишется со ...             0.968170  
2  В первом случае речь идет о названиях сельскох...             0.940848  
3  В слове "молоденьки

In [5]:
import pandas as pd
import re
from bs4 import BeautifulSoup

# Загрузка данных из CSV
file_path = 'RAG_ANSWERS.csv'  # Укажи путь к твоему файлу
df = pd.read_csv(file_path)

# Функция для очистки текста от HTML-тегов и ошибок кодировки
def clean_text(text):
    # Удаление HTML-тегов
    text = BeautifulSoup(text, "html.parser").get_text()
    # Удаление ошибок кодировки
    text = re.sub(r'&[a-zA-Z0-9#]+;', ' ', text)
    # Дополнительная очистка от лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Применение очистки к полю Correct_Answer
df['Correct_Answer_Cleaned'] = df['Correct_Answer'].apply(clean_text)
print(df['Correct_Answer_Cleaned'])
# Сохранение очищенного DataFrame в новый файл
df.to_csv('Cleaned_RAG_ANSWERS.csv', index=False)

# Вывод первых нескольких строк с очищенными данными
print(df[['Correct_Answer', 'Correct_Answer_Cleaned']].head())

0     Правильно слитное написание: Над головой хрипл...
1     В данном случае всероссийская пишется со строч...
2     Разница в типах текстов: такие названия не выд...
3     Вы путаете правило о написании суффиксов -еньк...
4     В терминологическом значении правильно только ...
5       Названия микрорайонов в кавычки не заключаются.
6                   Тире в данном предложении ставится.
7      Верно только инструкция по заполнению документа.
8     Правильно: «Обессиленный, я мешком повалился н...
9     Предпочтительно склонение, без выпадения гласн...
10              Точка перед запятой в сокращении нужна.
11                                       Запятая нужна.
12    Возможность существования в литературном русск...
13    Запятая нужна: здесь она ставится перед распро...
14                         Предпочтительно: чесменский.
15                     Да, необходимо тире с пробелами.
16                                Да, совершенно верно.
17    Инвест… — первая часть сложных слов, пишет